In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
from PIL import Image
from utils.options import Options
from utils.model_utils import load_models
from scripts.bald_proxy import BaldProxy
from scripts.align_proxy import AlignProxy
from utils.get_e4e import E4EEncoder
from utils.get_FS import FSEncoder
from scripts.blend_proxy import hairstyle_feature_blending
from utils.image_utils import show_results
%matplotlib inline

In [2]:
opts = Options().parse(jupyter=True)
generator, latent_avg, seg_net, kp_extractor, bald_mapper, alpha, bald_editor, sfe_encoder, align_mapper = load_models(opts)
e4e_encoder = E4EEncoder(opts.e4e_path, latent_avg, device=opts.device)
fs_encoder = FSEncoder(generator, latent_avg, sfe_encoder)
bald_proxy = BaldProxy(opts, generator, bald_mapper, alpha, seg_net, bald_editor, fs_encoder)
align_proxy = AlignProxy(opts, generator, seg_net, kp_extractor, latent_avg, align_mapper)

/root/autodl-tmp/conda/envs/sfe/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/root/autodl-tmp/conda/envs/sfe/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Loading ResNet ArcFace


In [ ]:
src_name = '000080.jpg'
ref_name = '089255.jpg'
src_pre = src_name.split('.')[0]
ref_pre = ref_name.split('.')[0]
src_path = os.path.join(opts.src_img_dir, src_name)
ref_path = os.path.join(opts.ref_img_dir, ref_name)
src_img = Image.open(src_path).convert('RGB')
ref_img = Image.open(ref_path).convert('RGB')
src_e4e_latent = e4e_encoder.get_e4e(src_img) 
inversion_img, src_sfe_w, src_sfe_F, _ = fs_encoder.get_FS(src_img) 
src_sfe_w = src_sfe_w.to(opts.device)
src_sfe_F = src_sfe_F.to(opts.device)
ref_e4e_latent = e4e_encoder.get_e4e(ref_img) 
src_sfe_w, w_bald, bald_feature, F_bald_edited, bald_img = bald_proxy(src_img, src_e4e_latent, src_sfe_w, src_sfe_F, possion=True)
edited_img, ref_rotated_w = align_proxy(ref_img, bald_img, w_bald, ref_e4e_latent, src_pre, ref_pre)
style_output = hairstyle_feature_blending(generator, seg_net, src_sfe_w, F_bald_edited, ref_rotated_w, bald_img)
style_output.save(os.path.join(opts.edited_img_dir, f'{src_pre}_{ref_pre}_edited.jpg'))
show_results(src_img, ref_img, bald_img, style_output)


Setting up PyTorch plugin "bias_act_plugin"...
Using /root/.cache/torch_extensions/py310_cu113 as PyTorch extensions root...
Detected CUDA files, patching ldflags
Emitting ninja build file /root/.cache/torch_extensions/py310_cu113/bias_act_plugin/build.ninja...
Building extension module bias_act_plugin...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
[1/3] /usr/local/cuda/bin/nvcc  -DTORCH_EXTENSION_NAME=bias_act_plugin -DTORCH_API_INCLUDE_EXTENSION_H -DPYBIND11_COMPILER_TYPE=\"_gcc\" -DPYBIND11_STDLIB=\"_libstdcpp\" -DPYBIND11_BUILD_ABI=\"_cxxabi1013\" -isystem /root/autodl-tmp/conda/envs/sfe/lib/python3.10/site-packages/torch/include -isystem /root/autodl-tmp/conda/envs/sfe/lib/python3.10/site-packages/torch/include/torch/csrc/api/include -isystem /root/autodl-tmp/conda/envs/sfe/lib/python3.10/site-packages/torch/include/TH -isystem /root/autodl-tmp/conda/envs/sfe/lib/python3.10/site-packages/torch/include/THC -isyst

"edited_img, ref_rotated_w = align_proxy(ref_img, bald_img, w_bald, ref_e4e_latent, src_pre, ref_pre)\nstyle_output = hairstyle_feature_blending(generator, seg_net, src_sfe_w, F_bald_edited, ref_rotated_w, bald_img)\nstyle_output.save(os.path.join(opts.edited_img_dir, f'{src_pre}_{ref_pre}_edited.jpg'))\nshow_results(src_img, ref_img, bald_img, style_output)"